**Autora** : Rafaela Ramos Sarmento

**e-mail**: rafaelaramos.datasci@gmail.com

#  <a name="resumo"> RESUMO </a>
[Voltar ao índice](#indice)

A Base de dados trabalhada é a **'Santander Customer Transaction Prediction'** disponível no link: https://www.kaggle.com/c/santander-customer-transaction-prediction

Essa base de dados envolve um problema de classificação binária, isto é, a partir das features disponíveis, decidir se caso é X ou Y, ou de maneira binária, 0 ou 1.


#  <a name="indice">  Índice </a>

* [Resumo](#resumo)
* [Índice](#indice)
* [Problema a ser analisado](#secao_0)
* [Análise descritiva e exploratória](#secao_1)
* [Random Forest](#secao_2)
* [Regressão Logística](#secao_3)
* [GBDT - XGBOOST](#secao_4)
* [Redes Neurais](#secao_5)
* [Comparação Entre  os Modelos](#secao_6)
* [Conclusão do Desafio](#secao_7) 

#  <a name="secao_0"> Problema a ser analisado </a>
[Voltar ao índice](#indice)

Como a base de dados escolhida foi a do **'Santander Customer Transaction Prediction'** temos o seguinte texto guia: 

   * "In this challenge, we invite Kagglers to help us identify which customers will make a specific transaction in the future, irrespective of the amount of money transacted. The data provided for this competition has the same structure as the real data we have available to solve this problem." *


Como temos um problema de classificação binária, e o próprio desafio sugere que a análise conste como uma previsão de que os clientes do banco realizem uma determinada operação no futuro, dadas as variáveis disponíveis no banco de dados do Santander, podemos formular um problema de negócio em que o banco estará oferecendo para o cliente um investimento de renda fixa do programa “Renda Mais”. Neste caso, os modelos de machine learning tem o objetivo de prever se um dado cliente irá participar do programa ou não.

Assim, as variáveis disponíveis poderiam ser interpretados como informações relacionadas a empréstimos realizados e pagos, pagamentos, idade, renda, situação civil e outras trasações e investimentos no banco.

Portanto, queremos criar um modelo de previsão que forneça o indicativo sobre quais clientes possuem maior probabilidade de fazer parte do programa "Renda Mais", de forma que os clientes classificados positivamente (1) receberão e-mails com propostas e propagandas referentes ao programa de investimento.

Para este caso, observa-se que falsos positivos (FP) não são um problema grave, uma vez que um cliente que seja classificado positivamente mas que não tenha interesse no programa poderá apenas ignorar ou recusar a oferta. Entretanto, o caso de falso negativo (FN) é considerado um problema grave, uma vez que o cliente classificado negativamente mas que tenha interesse em investir não recebera a oferta e, assim, o banco perde um cliente em potencial e não haverá lucros. A métrica de classificação de maior importância é o recall, de forma que maximizando o recall estaremos diminuindo o número de falsos negativos.
O recall é dado pela equação:


$$ recall = \frac{VP}{VP + FN}$$



O problema consta com a variável **ID_code** que é a identificação do cliente, a **target** que é o problema que estamos querendo resolver, isto é, é a variável do tipo classe (binária, 0 ou 1), e teremos um total de **200** features, identificadas como: var_0, var_1, ..., var_199.

Um detalhe é que a base de dados de teste não possui a coluna target, então não seria possível verificar as métricas do nosso modelo utilizando esses dados, portanto, o que será feito é dividir a base de treino em duas: uma efetivamente de treino e outra de validação, de forma a ter uma proporção 70% e 30%, respectivamente.
Além disso, a base de dados já esta bastante limpa, não sendo necessário fazer um trabalho arduo de pré-processamento dos dados.


#  <a name="secao_1"> Análise descritiva e exploratória </a>
[Voltar ao índice](#indice)

Vamos calcular algumas propriedades estatisticas dessa base de dados.

   - Não há valores nulos;
   - Dados em mesmo range de escala;
   - Classes desbalanceadas -> proporção 9 (0) para 1 (1)

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import statistics as sts
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.model_selection import train_test_split

In [4]:
#criando funcao para ter informacoes sobre os dados
def show_info(data) :
    print('DATASET SHAPE: ', data.shape, '\n')
    print('-'*50)
    print('FEATURE DATA TYPES:')
    print(data.info())
    print('\n', '-'*50)
    print('NUMBER OF UNIQUE VALUES PER FEATURE:', '\n')
    print(data.nunique())
    print('\n', '-'*50)
    print('NULL VALUES PER FEATURE')
    print(data.isnull().sum())
  

In [5]:
def plot_roc_curve(y_real, y_previsao, nome_modelo, color_roc = "pink") :
    rfp, rvp,lim = roc_curve(y_real,  y_previsao)
    # Gráfico da curva roc
    auc = roc_auc_score(y_real, y_previsao)
    plt.plot(rfp, rvp, marker='.', label='%s (AUC = %0.2f)' % (nome_modelo, auc),color=color_roc)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.title('Curva ROC - %s' % nome_modelo, fontsize=15)
    plt.xlabel('1- Especificidade', fontsize=12)
    plt.ylabel('Sensibilidade', fontsize=12)
    plt.grid(color='w', linestyle='dotted', linewidth=1)
    plt.legend()
    #plt.show()

In [6]:
def plot_curva_learning(modelo, nome_modelo) :
    results = modelo.evals_result()
    # plot learning curves
    plt.plot(results['validation_0']['logloss'], label='train')
    plt.plot(results['validation_1']['logloss'], label='test')
    plt.title('LogLoss vs n_estimator - %s' %nome_modelo)
    plt.ylabel('LogLoss')
    plt.xlabel('n_estimator')
    plt.grid(color='w', linestyle='dotted', linewidth=1)
    plt.legend()
    plt.show()

In [7]:
df = pd.read_csv("../santander-customer-transaction-prediction/train.csv")
df_train, df_validacao = train_test_split(df, train_size=0.7, shuffle=True)

#df_test = pd.read_csv("santander-customer-transaction-prediction/test.csv")

In [ ]:
#criando os arrays para X {variaveis previsoras} e Y {variavel tipo classe} para treino e teste
Y_train = df_train.iloc[:, 1].values
X_train = df_train.iloc[:, 2:].values
Y_validacao = df_validacao.iloc[:, 1].values 
X_validacao = df_validacao.iloc[:, 2:].values

In [ ]:
df_train.head()

In [ ]:
show_info(df_train)

In [ ]:
show_info(df_validacao)

In [ ]:
msno.bar(df_train)

#### Primeiras observações 

Legenda para entendimento do problema: 
   *  ID_code :  identificação do cliente
   *  target: variável que queremos prever
   *  var_i ; i = 0, 1, ..., 199
   
observa-se que em ambos conjuntos de dados, treino e validacao, não há valores nulos. Também não será necessário fazer processo de cleaning e escalonamento, visto que as variáveis estão sem significado e as ordens de grandeza estão próximas. 

Temos um caso de classes desbalanceadas, sendo as contagens:
  - Treino:
      * classe 0 : 125972 contagens  
      * classe 1 : 14028 contagens
  - Validação:
      * classe 0 : 53930 contagens  
      * classe 1 : 6070 contagens

In [ ]:
df_train.describe()

In [ ]:
df_validacao.describe()

In [ ]:
#verificando a quantidade de cada classe dentro do conjunto de dados de treino
sns.countplot(x=df_train['target'], palette = 'RdPu').set_title('target - treino')
plt.show()

In [ ]:
np.unique(Y_train, return_counts = True)

In [ ]:
#verificando a quantidade de cada classe dentro do conjunto de dados de validacao
sns.countplot(x=df_validacao['target'], palette = 'ocean').set_title('target - validação')
plt.show()

In [ ]:
np.unique(Y_validacao, return_counts = True)

In [ ]:
#calculo da matriz de correlação entre as variáveis do problema
plt.figure(figsize=(40,20)) 
plt.title('Correlação entre as features', size = 50) 
sns.heatmap(df_train.corr(), cmap='RdPu')
plt.show()

A seguir, o calculo matriz de correlacao, verifica quais entradas é maior que 0.5 e faz a contagem. A diagonal sempre vai valer 1, se tiver 201 contagens nao nulas, entao as variaveis nao sao correlacionadas.

In [ ]:
correlacao = df_train.corr()
qtd_correlacao = np.where(abs(correlacao)>0.5, 1, 0)
np.count_nonzero(qtd_correlacao), qtd_correlacao

#  <a name="secao_2"> Random Forest </a>
[Voltar ao índice](#indice)

Vamos iniciar o processo de previsão utilizando um algoritmo mais simples, o Random Forest.

Para este caso, foi testado alguns modelos manualmente e após isso foi utilizado o GridSearchCV() para encontrar o melhor modelo dentro de algumas possibilidades de hiper parâmetros. 

Com o melhor modelo, foi calculado as principais métricas para um caso de classificação, sendo elas a matriz de confusão, precision, recall e a curva ROC. Também foi plotado as 10 features mais importantes para o treino do modelo, essa informação poderia ser utilizada para eliminar features pouco importantes e deixar o treino com um custo computacional menor. Esse ultimo processo não foi feito nessa análise.


Parâmetros utilizados e variados:
   - n_estimator
   - max_depth
   - criterion = 'entropy' (fixo)
   
métrica maximizada: **recall** 

O melhor modelo, utilizando o GridSearchCV() foi construido usando os seguintes parâmetros:
   - n_estimator: 200
   - max_depth: 12

   
Com o recall sendo igual a:
   - Recall Score : 0.7235
   

In [ ]:
!pip install imblearn

In [ ]:
pip install threadpoolctl==3.1.0

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, recall_score, roc_auc_score
from yellowbrick.classifier import ConfusionMatrix
from imblearn.over_sampling import BorderlineSMOTE 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_roc_curve, RocCurveDisplay, roc_curve

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
X_validacao.shape, Y_validacao.shape

In [ ]:
#balanceando os dados, de forma a usar a técnica de oversampling
smote_random_forest = BorderlineSMOTE(sampling_strategy='minority')
X_over_train, Y_over_train = smote_random_forest.fit_resample(X_train, Y_train)
X_over_validacao, Y_over_validacao = smote_random_forest.fit_resample(X_validacao, Y_validacao)

In [ ]:
np.unique(Y_over_train, return_counts=True)

In [ ]:
np.unique(Y_over_validacao, return_counts=True)

In [ ]:
#inciando o treinamento com a Random Forest, usando a metrica para hierarquia das features
#a minimizacao da entropia (maximizacao do ganho de informacao)
random_forest = RandomForestClassifier(n_estimators=200, criterion='entropy', max_depth=12, verbose=1,random_state = 0)
random_forest.fit(X_over_train, Y_over_train)

In [ ]:
previsoes_random_forest = random_forest.predict(X_over_validacao)
recall_score(Y_over_validacao, previsoes_random_forest)

In [ ]:
#calculo de algumas métricas de classificação: matriz de confusao
matriz_confusao_random_forest = ConfusionMatrix(random_forest, cmap='GnBu')
matriz_confusao_random_forest.fit(X_over_train, Y_over_train)
matriz_confusao_random_forest.score(X_over_validacao, Y_over_validacao)

In [ ]:
#printa as métricas de classificação
print(classification_report(Y_over_validacao, previsoes_random_forest))

In [ ]:
#RocCurveDisplay.from_estimator(random_forest, X_over_validacao, Y_over_validacao, name = 'Random Forest')
roc_random_forest = plot_roc_curve(Y_over_validacao, previsoes_random_forest, 'Random Forest')
plt.show()

In [ ]:
#plotando o ranking de importancia das features utilizadas para treinar o meu modelo
feat_importances = pd.Series(random_forest.feature_importances_)
feat_importances.nlargest(10).plot(kind='barh')

In [ ]:
#tunando os hyperparametros com o gridsearch:
parametros = {'criterion': ['entropy'], 
              'n_estimators': [50, 100, 200],
              'max_depth': [6, 9, 12],
               'random_state': [0]}  

grid_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=parametros, scoring='recall', cv=2)
grid_rf.fit(X_over_train, Y_over_train)

In [ ]:
print('Melhor modelo: ' + str(grid_rf.best_estimator_))
print('Best Score: ' + str(grid_rf.best_score_))

#  <a name="secao_3"> Regressão Logística </a>
[Voltar ao índice](#indice)

Tentando um modelo mais simples para verificar como os dados se comportam. Para isso, será utilizado a regressão logística.

Parâmetros utilizados e variados:
   - max_iter
   - solver
   
métrica maximizada: recall 

O melhor modelo, utilizando o GridSearchCV() foi construido usando os seguintes parâmetros:
   - max_iter: 1500
   - solver: 'lbfgs'
   
Com o recall sendo igual a:
   - recall: 0.8037

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic_regression = LogisticRegression(max_iter=1500, random_state = 0)
logistic_regression.fit(X_over_train, Y_over_train)

In [ ]:
#logistic_regression.coef_

In [ ]:
previsoes_logistic = logistic_regression.predict(X_over_validacao)
recall_score(Y_over_validacao, previsoes_logistic)

In [ ]:
matriz_confusao_logistic = ConfusionMatrix(logistic_regression, cmap='GnBu')
matriz_confusao_logistic.fit(X_over_train, Y_over_train)
matriz_confusao_logistic.score(X_over_validacao, Y_over_validacao)

In [ ]:
print(classification_report(Y_over_validacao, previsoes_logistic))

In [ ]:
roc_logistic = plot_roc_curve(Y_over_validacao, previsoes_logistic, 'Logistic Regression')

In [ ]:
#tunando os hiperparametros da logistic regression com o GridSearchCV()
parametros_logistic = {'max_iter': [100, 300, 500, 1000, 1500],
                   'solver': ['lbfgs', 'saga'],
               }  

grid_logistic= GridSearchCV(estimator=LogisticRegression(), param_grid=parametros_logistic, cv=2, scoring='recall', verbose=0)
grid_logistic.fit(X_over_train, Y_over_train)

In [ ]:
print('Melhor modelo: ' + str(grid_logistic.best_estimator_))
print('Best Score: ' + str(grid_logistic.best_score_))

#  <a name="secao_4"> GBDT - XGBOOST </a>
[Voltar ao índice](#indice)

Utilizando a GBDT, um algoritmo mais avançado, em que teremos um ensemble de decisions trees trabalhando de forma sequencial. A GBDT aqui é implementada pela biblioteca XGBoost.

Parâmetros utilizados e variados:
   - n_estimator
   - max_depth
   - learning_rate

métrica maximizada: recall 

O melhor modelo, utilizando o GridSearchCV() foi construido usando os seguintes parâmetros:
   - n_estimator: 200
   - max_depth: 9
   - learning_rate: 0.3 
   
Com o recall sendo igual a:
   - recall: 0.7417

Entretanto, este modelo overfitou, assim, os parâmetros foram atualizados e adicionou-se um parâmetro de regularização (L2 - reg_gamma). De forma, que obteve-se o resultado:
   - n_estimator: 500
   - max_depth: 4
   - learning_rate: 0.1 
   
Com o recall sendo igual a:
   - recall: 0.8284

In [ ]:
import xgboost as xgb
import xgboost as get_score
from sklearn.metrics import log_loss

In [ ]:
#treinando modelo com os dados já balanceados pela etapa do random forest
GBDT = xgb.XGBClassifier(n_estimators=500, max_depth=4, learning_rate=0.1,random_state=0, reg_lambda=0.1)
GBDT.fit(X_over_train, Y_over_train, eval_metric='logloss', eval_set=[(X_over_train, Y_over_train), (X_over_validacao, Y_over_validacao)])

In [ ]:
previsoes_gbdt = GBDT.predict(X_over_validacao)
recall_score(Y_over_validacao, previsoes_gbdt)

In [ ]:
#calculo de algumas métricas de classificação: matriz de confusao
matriz_confusao_gbdt = ConfusionMatrix(GBDT, cmap='GnBu')
matriz_confusao_gbdt.fit(X_over_train, Y_over_train)
matriz_confusao_gbdt.score(X_over_validacao, Y_over_validacao)

In [ ]:
#printa as métricas de classificação
print(classification_report(Y_over_validacao, previsoes_gbdt))

In [ ]:
roc_gbdt = plot_roc_curve(Y_over_validacao, previsoes_gbdt, 'GBDT')

In [ ]:
logloss_gbdt = plot_curva_learning(GBDT, 'GBDT')

In [ ]:
feat_importances = pd.Series(GBDT.feature_importances_)
feat_importances.nlargest(10).plot(kind='barh')

In [ ]:
#tunando os hiperparametros da GBDT com o GridSearchCV()
parametros_gbdt = {'n_estimators': [100, 150, 200],
              'max_depth': [3, 6, 9],
                   'learning_rate': [0.1],
               }  

grid_gbdt = GridSearchCV(estimator=xgb.XGBClassifier(), param_grid=parametros_gbdt, cv=2, scoring='recall', verbose=0)
grid_gbdt.fit(X_over_train, Y_over_train)

In [ ]:
print('Melhor modelo: ' + str(grid_gbdt.best_estimator_))
print('Best Score: ' + str(grid_gbdt.best_score_))

#  <a name="secao_5"> Redes Neurais </a>
[Voltar ao índice](#indice)

Para a aplicação de redes neurais para resolver o problema, será utilizado uma rede neural simples, implementada pelo sklearn, a MLPClassifier()   (MultiLayer Perceptron).

Parâmetros utilizados e variados:
   - max_iter (epocas)
   - activation = 'logistic'
   - hidden_layer_sizes 
   
métrica maximizada: recall 

O melhor modelo encontrado:
   - max_iter (epocas) = 300 (efetivos=78)
   - hidden_layer_sizes = (100, 50, 25, 13,7) 
   
Com o recall sendo igual a:
   - recall: 0.7827
   

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
# 200 (neuronios entrada) -> 100 (1 camada oculta com 100 neuronios) -> 1 (neuronio saida)
rede_neural = MLPClassifier(max_iter=300, verbose=True, hidden_layer_sizes = (100, 50, 25, 13,7), activation='logistic', random_state=0)
rede_neural.fit(X_over_train, Y_over_train)

In [ ]:
previsoes_rn = rede_neural.predict(X_over_validacao)
recall_score(Y_over_validacao, previsoes_rn)

In [ ]:
matriz_confusao_rn = ConfusionMatrix(rede_neural, cmap='GnBu')
matriz_confusao_rn.fit(X_over_train, Y_over_train)
matriz_confusao_rn.score(X_over_validacao, Y_over_validacao)

In [ ]:
print(classification_report(Y_over_validacao, previsoes_rn))

In [ ]:
roc_redeneural = plot_roc_curve(Y_over_validacao, previsoes_rn, 'Rede Neural (MLP)')

#  <a name="secao_6"> Comparação Entre os Modelos </a>
[Voltar ao índice](#indice)   

Os modelos foram avaliados segundo a métrica **recall**. De forma que, resumidamente, obtivemos os seguintes resultados:

   -  **Random Forest**: 0.72
   - **Regressão Logistica**: 0.80
   - **GBDT**: 0.82
   - **Rede Neural**: 0.78    
   
Com isso, olhando apenas para o recall, a GBDT demonstrou possuir uma melhor performance, apesar de ter apresentado overfiting com o modelo retornado pelo GridSearchCV(). 
A GBDT também apresenta uma melhor curva ROC, estando mais localizada ao canto superior esquerdo do gráfico, sua AUC foi igual a 0.88.

In [ ]:
plot_roc_curve(Y_over_validacao, previsoes_random_forest, 'Random Forest', 'lightseagreen')
plot_roc_curve(Y_over_validacao, previsoes_logistic, 'Regressão Logística', 'darkolivegreen')
plot_roc_curve(Y_over_validacao, previsoes_gbdt, 'GBDT', 'red')
plot_roc_curve(Y_over_validacao, previsoes_rn, 'Rede Neural (MLP)', 'orange')
plt.show()

#  <a name="secao_7"> Conclusão do desafio </a>
[Voltar ao índice](#indice)

Como o problema proposto no desafio era bastante aberto, foi escolhido uma abordagem exploratória de modelos de Machine Learning para realizar a previsão de uma possível tranasação envolvendo clientes do banco Santander. O problema envolveu um banco de dados do Kaggle e consistiu de um problema de classificação binária, também preferido pelo enunciado do desafio. 

O pré processamento foi curto uma vez que o dados eram bastante limpos e comportados, bastando apenas de um processo de balanceamento, no qual se usou uma técnica de oversampling. A análise preditiva de classificação buscou testar desde os modelos mais simples, como a Regressão Logística, até os mais sofisticados, como o GBDT. O que se observou foi que apesar do nivel de complexidade entre estas duas técnicas ser bastante diferente, ambas retornaram um valor de recall bastante próximos, de forma que o custo-benefício tende a Regressão Logística. 

O pior valor encontrado foi referente ao Random Forest.

A verificação de overfitting foi realizada apenas para o GBDT, mas poderia ter sido feito para todos os demais modelos. 